# Data ADS

In [7]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

import matplotlib.pyplot as plt

In [8]:
# 1. Đọc danh sách ASIN từ file txt
with open('ASIN.txt', 'r') as f:
    asin_list = [line.strip() for line in f.readlines()]

# 2. Tạo list chứa các file .xlsx trong thư mục Data
folder_path = 'Data_ADS'
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# 3. Đọc từng file, xử lý rồi append vào list
all_data = []

for file in files:
    # Trích xuất ngày từ tên file
    date_str = file.split('_')[1].replace('.xlsx', '')  # "20250713"
    date_obj = datetime.strptime(date_str, "%Y%m%d")     # datetime object
    df = pd.read_excel(os.path.join(folder_path, file))
    
    # Thêm cột date định dạng chuẩn
    df['date'] = date_obj
    
    # Tạo cột ASIN từ 10 ký tự đầu tiên của cột Portfolio
    df['ASIN'] = df['Portfolio'].astype(str).str[:10]
    
    all_data.append(df)

# 4. Gộp tất cả thành một DataFrame
merged_df = pd.concat(all_data, ignore_index=True)

# 5. Lọc chỉ giữ lại các dòng có ASIN nằm trong danh sách
filtered_df = merged_df[merged_df['ASIN'].isin(asin_list)]

#6. Di chuuyển cột ASIN lên đầu, sau Campaign type	
filtered_df = filtered_df[['Campaign type', 'ASIN'] + [col for col in filtered_df.columns if col not in ['Campaign type', 'ASIN']]]

# ✅ Kết quả cuối cùng
filtered_df.head()

,Campaign type,ASIN,Campaign,Status,Country,Profile,Portfolio,Target type,Daily Budget,Current Budget,...,ACOS,ROAS,CPA,Sales Same SKU,Sales Other SKU,Orders Same SKU,Orders Other SKU,Units Same SKU,Units Other SKU,date
27,sponsoredProducts,B08R8R2LQF,B08R8R2LQF_20oz_May the forties black_40th bir...,Out Of Budget,US,NewEleven,B08R8R2LQF_TUMBLER 20_MAY THE FORTIES BE WITH ...,manual,$10.00,$0.00,...,1.0379,0.96,23.85,22.98,0.00,1,--,1,--,2025-07-27
31,sponsoredProducts,B0DH86X93V,B0DH86X93V_20oz_vintage 1985 weird navy_40th b...,Delivering,US,NewEleven,B0DH86X93V_TUMBLER 20_VINTAGE 1985 WEIRD BEING...,manual,$24.00,$0.00,...,0.3713,2.69,7.79,19.98,42.96,1,2,1,2,2025-07-27
56,sponsoredProducts,B0F2T4JDF7,B0F2T4JDF7_Card_Youre awesome mom_birthday car...,Delivering,US,NewEleven,B0F2T4JDF7_CARD_YOU'RE AN AWESOME MOM_TRINH,manual,$10.00,$0.00,...,0.4805,2.08,4.80,39.96,0.00,4,--,4,--,2025-07-27
60,sponsoredProducts,B092HNBJT6,B092HNBJT6_20oz_Coworker black_auto 15h40 (max...,Out Of Budget,US,NewEleven,B092HNBJT6_TUMBLER 20_COWORKER BLACK_TRINH,auto,$10.00,$0.00,...,0.3153,3.17,6.30,0.00,59.94,0,3,0,3,2025-07-27
72,sponsoredProducts,B09XXP5V2W,"B09XXP5V2W_20oz_Quitter black_retirement gift_b,p",Out Of Budget,US,NewEleven,B09XXP5V2W_TUMBLER 20_QUITTER BLACK_TRINH,manual,$10.00,$0.00,...,--,0,0.00,0.00,0.00,0,--,0,--,2025-07-27


In [9]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/ppcintern/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1n2Ug3joJwsuuSsrx6T92Zi_dihht7KzIebRTGpKnehc"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("DATA_XN_ADS")

set_with_dataframe(sheet1, filtered_df)

# Total GMV

In [15]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

import matplotlib.pyplot as plt

In [16]:
# 1. Đọc danh sách ASIN từ file txt
with open('ASIN.txt', 'r') as f:
    asin_list = [line.strip() for line in f.readlines()]

# 2. Tạo list chứa các file .csv trong thư mục Data
folder_path = 'Data_Total'
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 3. Đọc từng file, xử lý rồi append vào list
all_data = []

for file in files:
    # ✅ Dùng regex để tìm ngày theo pattern dd_mm_yyyy
    match = re.search(r'(\d{2}_\d{2}_\d{4})', file)
    
    if match:
        date_str = match.group(1)  # ví dụ: "20_07_2025"
        date_obj = datetime.strptime(date_str, "%d_%m_%Y")  # chuyển thành datetime object

        df = pd.read_csv(os.path.join(folder_path, file), sep=";")
        
        # Thêm cột 'Date' chuẩn hóa theo yyyy-mm-dd
        df['Date'] = date_obj.strftime('%Y-%m-%d')
        
        all_data.append(df)
    else:
        print(f"⚠️ Không tìm thấy ngày trong tên file: {file}")

# 4. Gộp tất cả thành một DataFrame
merged_df = pd.concat(all_data, ignore_index=True)

# ✅ Kết quả
print(merged_df.head())

                                             Product        ASIN  \
0  NewEleven Retirement Gifts For Men 2025 - Cowo...  B0F18CMJ3L   
1  NewEleven 50th Birthday Gifts For Men Women - ...  B0DH7YGD9Q   
2  NewEleven Gifts For Aunt - Aunt Gifts From Nie...  B0CQJZP6TK   
3  NewEleven 40th Birthday Gifts For Women, Her -...  B0DGGDX9P5   
4  NewEleven Retirement Gifts, Farewell Gift, Goo...  B0D7HMXCSP   

            SKU  Units  Refunds     Sales  Promo      Ads  \
0  CU-4EWE-9UH1   51.0      NaN  1 171,98  -1,61  -157,38   
1  EC-NYJH-0YZA   42.0      4.0    839,16   -8,2  -215,28   
2  FJ-K5JP-AWYT   30.0      1.0     479,4  -9,12   -63,96   
3  EI-6RN9-0Z0A   27.0      1.0    458,46  -0,85  -174,62   
4  UB-TA3P-4YBQ   26.0      2.0    415,48    NaN      NaN   

  Sponsored products (PPC)  Sponsored Display  ...  Net profit  \
0                  -157,38                NaN  ...      487,48   
1                  -215,28                NaN  ...      101,44   
2                   -63,96

In [17]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/ppcintern/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1n2Ug3joJwsuuSsrx6T92Zi_dihht7KzIebRTGpKnehc"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("DATA_SB_TOTAL")

sheet1.clear()  # Xóa dữ liệu cũ trước khi ghi mới
set_with_dataframe(sheet1, merged_df)